# Running on QubiC Hardware

This notebook shows how to use pynq-quantum with the **QubiC** backend, which drives RFSoC hardware through the QubiC compiler and JSON configuration system.

**Prerequisites:**
- A Xilinx RFSoC board with QubiC firmware
- The `qubic` Python package installed
- A QubiC JSON configuration file describing channel mappings

> **Note:** This notebook is illustrative. The code below will not run without QubiC hardware and a valid configuration file.

In [ ]:
# Requires QubiC hardware -- will raise ImportError or FileNotFoundError
# on machines without the QubiC toolchain.

from pynq_quantum import QuantumOverlay, QubitController

# Load the QubiC backend with a JSON config file
# Replace config_path with your actual QubiC configuration
overlay = QuantumOverlay(
    backend="qubic",
    config_path="/path/to/qubic_config.json",
)

print(f"Backend: {overlay.backend_name}")
print(f"Channels: {overlay.backend.num_channels}")

In [ ]:
# Configure qubit parameters
ctrl = QubitController(overlay, num_qubits=2)
ctrl.set_qubit_frequency(0, 5.10e9)   # Qubit 0 at 5.10 GHz
ctrl.set_qubit_frequency(1, 4.95e9)   # Qubit 1 at 4.95 GHz

# Run a simple X gate on qubit 0
ctrl.x(0)
ctrl.measure()
result = ctrl.run(shots=1000)

print(f"X gate on QubiC hardware: {result.counts}")

overlay.close()

## QubiC Configuration File

The QubiC backend expects a JSON configuration file with at minimum a `channels` section. Example structure:

```json
{
  "channels": {
    "0": {"type": "drive", "dac": 0, "frequency": 5.1e9},
    "1": {"type": "drive", "dac": 1, "frequency": 4.95e9},
    "2": {"type": "readout", "adc": 0, "frequency": 6.8e9}
  },
  "compiler": {},
  "max_shots": 100000,
  "sample_rate": 6.144e9
}
```

The QubiC compiler translates pulse instructions into low-level FPGA instructions, uploads them, and acquires measurement data.